# InfluxDB Testing

In [24]:
from influxdb_client import InfluxDBClient
import dotenv
import os
import datetime

In [13]:
dotenv.load_dotenv('.env')
url = os.getenv("INFLUXDB_URL")
token = os.getenv("INFLUXDB_TOKEN")
org = os.getenv("INFLUXDB_ORG")
bucket = os.getenv("INFLUXDB_BUCKET")

In [32]:
query = f'from(bucket:"{bucket}") |> range(start: -12m, stop: -2m) \
  |> filter(fn: (r) => r["entity_id"] == "sonoff_1001e01c1e_power")\
  |> filter(fn: (r) => r["_field"] == "value")\
  '

client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()
tables = query_api.query(query=query)

In [33]:
measurement = []
listoflist = []
i = 0
for table in tables:
    for row in table.records:
        record = [row.values["_time"].timestamp(), row.values["_value"]]
        listoflist.append(record)


In [ ]:
listoflist

In [35]:
# define static variables
utid = round(datetime.datetime.now().timestamp())
cstate='C7'
hrs = datetime.datetime.now().hour
cpu = 1.88 # will be actual function to get cpu usage in linux environment


In [ ]:
with open('influx-data.csv', 'w') as f:
    f.write('utid,C-state,hrs,time,value,cpu\n') # write header

In [36]:
for i in listoflist:
    with open('influx-data.csv', 'a') as f:
        f.write(f'{utid},{cstate},{hrs},{i[0]},{i[1]},{cpu}\n')

Generating Random Data for Analysis

In [111]:
import random

random_int_c7 = [random.randint(2200,3000)/100 for i in range(200)]
random_int_c3 = [random.randint(2250,3000)/100 for i in range(200)]
random_int_c0 = [random.randint(2300,3000)/100 for i in range(200)]

In [112]:
def write_random(c, rand_list, writer):
    hrs = 9
    for i in rand_list:
        hrs += 1
        writer.write(f'Timestamp, {hrs}, {i}, {c}\n')


with open('influx-test.csv','w') as csvfile:
    csvfile.write('Timestamp, Hrs, Value, C-State\n')
    write_random('C7', random_int_c7, csvfile)
    write_random('C3', random_int_c3, csvfile)
    write_random('C0', random_int_c0, csvfile)
